In [82]:
from typing import Iterator
from agno.agent import Agent, RunResponse
from agno.models.openai import OpenAIChat
from agno.utils.pprint import pprint_run_response
from pydantic import BaseModel, Field
from agno.storage.sqlite import SqliteStorage
from agno.team.team import Team
import requests
import httpx
from agno.tools import tool
import os
import uuid
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs
from typing import IO
from io import BytesIO


In [83]:
from dotenv import load_dotenv

In [84]:
load_dotenv()

True

In [36]:
import psycopg2

def get_connection():
    """
    Establishes and returns a connection to the PostgreSQL database.
    Update the connection details as per your environment.
    """
    conn = psycopg2.connect(
        host="localhost",
        port=5432,
        dbname="phidata_db",
        user="postgres",
        password="postgres"
    )
    return conn

In [37]:
@tool(name="get_table_schema",                
    description="Fetches the schema of the table.", 
    show_result=True)
def get_table_schema() -> str:
    """
    Retrieves the schema for the given table name from the PostgreSQL
    information_schema and returns a formatted string listing each column,
    its data type, and (if applicable) its character maximum length.
    """
    conn = get_connection()
    table_name= "vehicle_policies"
    try:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT column_name, data_type, character_maximum_length
                FROM information_schema.columns
                WHERE table_name = %s
                ORDER BY ordinal_position;
            """, (table_name,))
            rows = cur.fetchall()
            if not rows:
                return f"No schema found for table '{table_name}'."
            
            schema_lines = []
            for column_name, data_type, char_max_length in rows:
                if char_max_length:
                    schema_lines.append(f"{column_name}: {data_type}({char_max_length})")
                else:
                    schema_lines.append(f"{column_name}: {data_type}")
            return "\n".join(schema_lines)
    finally:
        conn.close()

In [50]:
@tool(name="execute_sql",                
    description="Executes the SQL in the database and fetches the vehicle policy related information ", 
    show_result=True)
def execute_sql_query(query: str) -> str:
    """
    Executes the provided SQL query in PostgreSQL and returns the vehicle policy results
    as a string.
    """
    conn = get_connection()
    try:
        with conn.cursor() as cur:
            cur.execute(query)
            if cur.description:
                # This is a SELECT (or similar) query returning rows.
                rows = cur.fetchall()
                return "\n".join(str(row) for row in rows)
            else:
                # For queries like INSERT, UPDATE, or DDL commands
                conn.commit()
                return "Query executed successfully."
    except Exception as e:
        return f"Error executing query: {e}"
    finally:
        conn.close()

In [62]:
@tool(name="fetch_claim_status",                
    description="Fetches the status of the claim based on the claim id", 
    show_result=True)
def fetch_claim_status(claim_id):
    payload = {"claim_id": claim_id}
    status = requests.post("http://127.0.0.1:8000/claim/status", json=payload)
    return "The status of the claim is : "+str(status.json()['status'])

In [63]:
@tool(name="fetch_claim_date",                
    description="Fetches the date on which the claim is raised based on the claim id", 
    show_result=True)
def fetch_claim_date(claim_id):
    payload = {"claim_id": claim_id}
    date = requests.post("http://127.0.0.1:8000/claim/date", json=payload)
    return "The Date on which the claim has been done is : "+ str(date.json()['claim_date'])


In [64]:
@tool(name="fetch_latest_comment_on_claim",                
    description="Fetches the latest comment by the insurers on the claim based on the claim id", 
    show_result=True)
def fetch_latest_comment_on_claim(claim_id):
    payload = {"claim_id": claim_id}
    comment = requests.post("http://127.0.0.1:8000/claim/comment", json=payload)
    return "The Latest Comment Done by the insurer on my claim is : " + str(comment.json()['latest_comment'])


In [65]:
sql_generator_agent = Agent(
    name="SQL_Query_Generator",
    role="An SQL Query Generator",
    model=OpenAIChat(id="gpt-4o"),
    tools=[get_table_schema],
    instructions=[f"You are an expert SQL query generator. Using the provided tools, craft a precise and valid SQL query that directly addresses the user's question. Rely solely on the schema details to ensure the query is accurate and complete."],
    show_tool_calls=True,
    markdown=True,
)

In [66]:
# sql_generator_agent.run("what is the policy_number for vehicle TS08AX5678 ")

In [67]:
sql_exec_agent = Agent(
    name="SQL Executor Agent",
    role="Agent for Executing SQL Queries and Translating Database Results",
    model=OpenAIChat(id="gpt-4o"),
    tools=[execute_sql_query],
    instructions=[f"You are an agent designed to receive SQL queries as input, execute them using the provided database tool, and then convert the raw query results into clear, natural language explanations. Your responses should be both accurate and easy for non-technical users to understand. Dont mention/explain anything about SQL Query provided"],
    show_tool_calls=True,
    markdown=True,
)

In [68]:
api_agent = Agent(
    name="API Trigger Agent",
    role="Handles user queries related to insurance claims by invoking the appropriate API tools to retrieve claim status, submission date, or the latest insurer comments.",
    model=OpenAIChat(id="gpt-4o"),
    tools=[fetch_latest_comment_on_claim, fetch_claim_status, fetch_claim_date],
    instructions=[
        "Understand the user's intent regarding claim-related information.",
        "Based on the query, trigger the relevant tool to fetch claim details such as status, submission date, or insurer's latest comment.",
        "Use the provided tools only. Do not fabricate responses or assume values."
    ],
    show_tool_calls=True,
    markdown=True,
)

In [69]:
sql_team = Team(
    name="SQL Generator Executor Team",
    mode="coordinate",
    model=OpenAIChat("gpt-4.5-preview"),
    members=[
        sql_generator_agent,
        sql_exec_agent
    ],
    show_tool_calls=True,
    markdown=True,
    instructions=[
        "Begin by using the SQL Query Generator to create a valid SQL statement based on the user's request and the schema provided.",
        "Next, forward the generated SQL to the SQL Executor, which will run the query and interpret the results.",
        "Finally, present a clear, user-friendly summary of the outcome—without exposing any SQL syntax or internal details."
    ],
    show_members_responses=True,
)

In [70]:
final_team = Team(
    name="User Query Response Team",
    mode="route",
    description=(
        "You are a precise and reliable classifier working in the insurance domain." 
        "Your job is to analyse the user query and naviage to the respective agent"
    ),
    model=OpenAIChat("gpt-4.5-preview"),
    members=[
        sql_team,
        api_agent
    ],
    show_tool_calls=True,
    markdown=True,
    instructions=[
        "based on the user query u can navigate/ trigger either the sql agent or the api agent "
    ],
    show_members_responses=True,
)

In [73]:
final_team.run("what is the policy_number for vehicle TS08AX5678")

TeamRunResponse(event='RunResponse', content='The policy number associated with vehicle TS08AX5678 is **POLICY2024PRIYA**.', content_type='str', thinking=None, messages=[Message(role='system', content="You are the leader of a team of AI Agents and possible Sub-Teams:\n - Team: SQL Generator Executor Team\n   - Agent 1:\n     - Name: SQL_Query_Generator\n     - Role: An SQL Query Generator\n     - Available tools:\n      - get_table_schema: Retrieves the schema for the given table name from the PostgreSQL\ninformation_schema and returns a formatted string listing each column,\nits data type, and (if applicable) its character maximum length.\n   - Agent 2:\n     - Name: SQL Executor Agent\n     - Role: Agent for Executing SQL Queries and Translating Database Results\n     - Available tools:\n      - execute_sql: Executes the provided SQL query in PostgreSQL and returns the vehicle policy results\nas a string.\n - Agent 2:\n   - Name: API Trigger Agent\n   - Role: Handles user queries rel

In [72]:
final_team.run("what is the status of the claim CLM12345 ?")

TeamRunResponse(event='RunResponse', content='The status of the claim is : ApprovedThe status of the claim CLM12345 is: **Approved**.', content_type='str', thinking=None, messages=[Message(role='system', content="You are the leader of a team of AI Agents and possible Sub-Teams:\n - Team: SQL Generator Executor Team\n   - Agent 1:\n     - Name: SQL_Query_Generator\n     - Role: An SQL Query Generator\n     - Available tools:\n      - get_table_schema: Retrieves the schema for the given table name from the PostgreSQL\ninformation_schema and returns a formatted string listing each column,\nits data type, and (if applicable) its character maximum length.\n   - Agent 2:\n     - Name: SQL Executor Agent\n     - Role: Agent for Executing SQL Queries and Translating Database Results\n     - Available tools:\n      - execute_sql: Executes the provided SQL query in PostgreSQL and returns the vehicle policy results\nas a string.\n - Agent 2:\n   - Name: API Trigger Agent\n   - Role: Handles user 

In [78]:
res = final_team.run("what is the status of the claim CLM12345 ?")
response =  res.content

In [79]:
print(response)

The status of the claim is : ApprovedThe status of the claim CLM12345 is **Approved**.


In [85]:
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
client = ElevenLabs(
    api_key=ELEVENLABS_API_KEY,
)

In [87]:
def text_to_speech_stream(text: str) -> IO[bytes]:
    # Perform the text-to-speech conversion
    response = client.text_to_speech.convert(
        voice_id="pNInz6obpgDQGcFmaJgB", # Adam pre-made voice
        output_format="mp3_22050_32",
        text=text,
        model_id="eleven_multilingual_v2",
        # Optional voice settings that allow you to customize the output
        voice_settings=VoiceSettings(
            stability=0.0,
            similarity_boost=1.0,
            style=0.0,
            use_speaker_boost=True,
            speed=1.0,
        ),
    )
    # Create a BytesIO object to hold the audio data in memory
    audio_stream = BytesIO()
    # Write each chunk of audio data to the stream
    for chunk in response:
        if chunk:
            audio_stream.write(chunk)
    # Reset stream position to the beginning
    audio_stream.seek(0)
    # Return the stream for further use
    return audio_stream

In [89]:
from IPython.display import Audio

stream = text_to_speech_stream(response)
Audio(stream.read(), autoplay=True)

In [88]:
text_to_speech_stream(response)